In [1]:
import numpy as np

In [6]:
from sklearn.metrics import average_precision_score

In [ ]:
class BboxMAP(object):
    def __init__(self):
        self.y_trues = {}
        self.y_scores = {}
        
    def update_scores(self,net_output):
        #self.y_trues...
        #self.y_scores...
        pass
    
    def _compute_score(net_output, label):
        
        pass
    
    def filter_out_zeros(self):
        pass
 

    def compute_final_accuracy(self):
        y_true, y_score = get_ytrue_yscore()
        APs = compute_final_APs(y_true, y_score)
        mAP = compute_mAP(aps)
        return mAP


    def compute_mAP(self,APs):
        return np.mean(APs.values())

    def compute_final_APs(self):
        APs = {}
        for cls in y_score.keys():
            APs[cls] = average_precision_score(self.y_true[cls], self.y_score[cls])
        return APs
    
    

In [314]:
def unparametrize(xy,wh, scale_factor=32):
    xy_raw = unparametrize_xy(xy,scale_factor)
    wh_raw = unparametrize_wh(wh,scale_factor)
    return xy_raw, wh_raw
    

def unparametrize_xy(xy,scale_factor):
    #xy should be num_ex,2,24,24
    num_ex = xy.shape[0]
    xdim = xy.shape[-2]
    ydim = xy.shape[-1]
    xarange = np.arange(xdim)
    yarange = np.arange(ydim)
    xgrid, ygrid = np.meshgrid(xarange, yarange)
    grid = np.vstack((xgrid, ygrid)).reshape(2,xdim,ydim)
    inds = np.zeros_like(xy) # np.repeat(grid,[num_ex,num_ex],0)#.reshape(num_ex,2,xdim,ydim)
    inds[:,0] = xgrid
    inds[:,1] = ygrid
    
    #xy is the normalized offsets, so we add by the ind to get unoffset, then multiply by scale factor to get raw
    raw_normalized = xy + inds
    xy_raw = scale_factor * raw_normalized
    return xy_raw
    
    

def unparametrize_wh(wh,scale_factor):
    wh_unlog = np.power(2*np.ones_like(wh), wh)
    wh_raw = scale_factor * wh_unlog
    return wh_raw
    
    
def convert_from_xy_wh_to_min_max(xy_raw,wh_raw):
        
        x = xy_raw[:,0]
        y = xy_raw[:,1]
        w = wh_raw[:,0]
        h = wh_raw[:,1]
        xmin,xmax = np.maximum(np.zeros_like(x),x - w / 2.), x + w/ 2.
        ymin, ymax = np.maximum(np.zeros_like(y),y - h / 2.), y+ h/ 2.
        xmin, xmax, ymin, ymax = [np.expand_dims(arr,axis=1) for arr in [xmin, xmax, ymin, ymax]]
        return np.concatenate((xmin, xmax, ymin, ymax),axis=1)
    

In [315]:
def test_unparametrize():
    xy = np.random.rand(4,2,24,24)

    xy_raw = unparametrize_xy(xy, 32)

    abs_range = np.abs(np.log2(1./32)) + np.log2(24)
    small = np.log2(1./32)

    wh = abs_range* np.random.rand(4,2,24,24) + small

    wh_raw = unparametrize_wh(wh,32)
    assert np.min(wh_raw) > 0.
    assert np.max(wh_raw) < 768.
    assert np.min(xy_raw) > 0.
    assert np.max(xy_raw) < 768.
    return xy_raw, wh_raw

In [316]:
xy_raw,wh_raw = test_unparametrize()

In [317]:
def test_convert():
    xy_raw,wh_raw = test_unparametrize()
    assert np.all(xy_raw >= 0.0)
    assert np.all(wh_raw >= 0.0)
    return convert_from_xy_wh_to_min_max(xy_raw,wh_raw)
    

In [136]:
def iou(xmin_max_y_min_max_pred, xmin_max_y_min_max_gt):
    #xmin_max_y_min_max_pred -> both are shapes: num_ex,4,24,24  4 channels are (xmin, xmas,ymin,ymax)
    # return iou -> num_ex,24,24 -> iou for all places

    XMIN, XMAX, YMIN,YMAX = range(4)

    x_max_pred, x_max_gt = xmin_max_y_min_max_pred[:,XMAX],xmin_max_y_min_max_gt[:,XMAX]
    x_min_pred, x_min_gt = xmin_max_y_min_max_pred[:,XMIN],xmin_max_y_min_max_gt[:,XMIN]
    y_max_pred, y_max_gt = xmin_max_y_min_max_pred[:,YMAX],xmin_max_y_min_max_gt[:,YMAX]
    y_min_pred, y_min_gt = xmin_max_y_min_max_pred[:,YMIN],xmin_max_y_min_max_gt[:,YMIN]


    #min of the xmaxes
    min_xmax = np.minimum(x_max_pred, x_max_gt )

    #max of the xmins
    max_xmin = np.maximum(x_min_pred, x_min_gt )

    #min of the ymaxes
    min_ymax = np.minimum(y_max_pred, y_max_gt )

    #max of the ymins
    max_ymin = np.maximum(y_min_pred, y_min_gt )

    xdiff = min_xmax - max_xmin

    ydiff = min_ymax - max_ymin

    inters = np.maximum(np.zeros_like(xdiff), xdiff) * np.maximum(np.zeros_like(ydiff), ydiff)

    def get_area(x_min, x_max, y_min, y_max):
        area = (x_max - x_min) * (y_max - y_min)
        assert np.all(area > 0.0), "area should be greater than zero!"
        return area

    pred_area = get_area(x_min_pred, x_max_pred, y_min_pred, y_max_pred)
    gt_area = get_area(x_min_gt, x_max_gt, y_min_gt, y_max_gt)

    union = pred_area + gt_area - inters
    iou = inters / union.astype(np.float32)
    return iou

In [137]:
def test_iou():
    def make_test_array(box1,box2):

        test = np.array([box1, box2]).transpose((1,0)).reshape(4,2,1)

        test = np.concatenate((test,test), axis=2)
        test = np.vstack((test,test)).reshape(2,4,2,2)
        return test
    test_pred = make_test_array([0,2,0,3],[3,5,1,3])

    test_gt = make_test_array([1,3,1,4], [3,5,2,5])

    test_answer = np.asarray([[0.2,0.2],[0.25,0.25]])

    test_answer = np.vstack((test_answer, test_answer)).reshape(2,2,2)
    assert np.allclose(iou(test_pred, test_gt), test_answer)

In [139]:
test_iou()

In [ ]:
# xy_pred (4, 2, 24, 24)
# wh_pred (4, 2, 24, 24)
# xy_gt (4, 2, 24, 24)
# wh_gt (4, 2, 24, 24)
# class_scores_copy (4, 4, 24, 24)
# obj_scores_copy (4, 2, 24, 24)
# gcls (4, 1, 24, 24)
# gobj (4, 1, 24, 24)